## Add additional requirements locally

In [ ]:
#!pip install --user -I toml pandas dask

In [ ]:
#!pip install --upgrade pandas numpy dask black[jupyter] uproot3 pyarrow astropy toml

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.colors
from matplotlib.colors import LogNorm
import pandas
# import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)
#import cdms
import sys
import scipy
import random
import scipy.signal
import numpy as np
# from importlib import reload

In [ ]:
import dask.dataframe as dd
import dask.array as da

## Autoreload `cdat`

In [ ]:
from cdat.root import read_root_file
from cdat.daskutils import dask_histogram, dask_histogram2d

In [ ]:
# !ls '/cvmfs/data/CDMS/RQanalysis_testing/RQroot/'

In [ ]:
# Finding address of the files
data_dir = '/cvmfs/data/CDMS/RQanalysis_testing/RQroot/'
series_numbers =['25220213_171932','25220214_092356','25220215_073923']
files = [f'{data_dir}OFResults_{i}.root' for i in series_numbers]

In [ ]:
# this loads the first file to inspect structure of the dataframe
dask_dataframe = dd.from_map(read_root_file, files,branches=["*OFL*","*trig*",'*Integral*','*file*','*Midas*'])

In [ ]:
dask_dataframe.head(100000)

In [ ]:
%%time

KeV_bin = {'NFC1':22.9,'NFH':7.1,'NFE':5.7,'NFC2':17.1}
live_time = 44.85 #h
det_mass = 0.96 #g

data = dd.from_map(read_root_file, files,branches=["*trig_ch*",'*Integral*_total'])
trigger_ch = {'NFC1':1,'NFH':2,'NFE':3,'NFC2':4}
#For different detectors, building histogram domains separately.
xedges = {}
chunk_hist = {}
centers= {}
bin_sizes = {}
for det, code in trigger_ch.items():
    if det in ['NFC1','NFC2']:
        start, end = 0,8000
        step = (end-start)/1000
        xedges[det] = np.arange(start,end,step)  
    if det == "NFH":
        start, end = 0,2500
        step = (end-start)/1000
        xedges[det] = np.arange(start,end,step)
    if det == "NFE":
        start, end = 0,2000
        step = (end-start)/1000
        xedges[det] = np.arange(start,end,step)
    #Empty arrays for each channel corresponding to xedges.
    # We fill the empty array as we iterate through the data.
    centers[det] = (xedges[det][:-1]+xedges[det][1:])/2
    bin_sizes[det] = xedges[det][1:] - xedges[det][:-1]    

#iterating over dets
histograms = []
for det, code in trigger_ch.items():
    #Applying some inline cuts
    subchunk = data.query(f'trig_ch == {code}')
    #Filling numpy arrays for histograms.
    histograms.append(dask_histogram(subchunk, f'Integral_{det}_total', bins=xedges[det], bins_range=None)[1])

histograms = da.compute(*histograms, num_workers=4)
# turn list into dict {ch:hist[ch]}
chunk_hist = dict(zip(centers.keys(), histograms))
#plotting histograms.
fig,ax = plt.subplots(2,2,figsize=(16,16))
for det,code in trigger_ch.items():
    i , j = (code-1)//2 , code%2 
    ax[i,j].step(centers[det],chunk_hist[det]/(det_mass*live_time*bin_sizes[det]))
    ax[i,j].set_xlabel(f"Integral_{det}_total (uA)",fontsize = 20)
    ax[i,j].set_ylabel(r'Count/($gram.hour.bin$)',fontsize = 20)
    ax[i,j].set_yscale('log')
    ax[i,j].tick_params(axis='both', which='both', labelsize=15)

In [ ]:
data = dd.from_map(read_root_file, files,branches=["*OFL*","*trig_ch*",'*Integral*_total'])

trigger_ch = {'NFC1':1,'NFH':2,'NFE':3,'NFC2':4}
KeV_bin = {'NFC1':22.9,'NFH':7.1,'NFE':5.7,'NFC2':17.1}
auto_proc_high_energy = {'NFE':0.103, 'NFH':0.049, 'NFC1':0.129,'NFC2':0.0796}
live_time = 44.85 #h
det_mass = 0.96 #g


#For different detectors, building histogram domains separately.

xedges = {}
yedges = {}

chunk_hist = {}

for det, code in trigger_ch.items():
    if det in ['NFC1','NFC2']:
        xstart, xend = 0,900
        xstep = (xend-xstart)/100
        xedges[det] = np.arange(xstart,xend,xstep)
        
        ystart, yend = 0,80
        ystep = (yend-ystart)/100
        yedges[det] = np.arange(ystart,yend,ystep)
        
    if det == "NFH":
        xstart, xend = 0,900
        xstep = (xend-xstart)/100
        xedges[det] = np.arange(xstart,xend,xstep)
        ystart, yend = 0,80
        ystep = (yend-ystart)/100
        yedges[det] = np.arange(ystart,yend,ystep)
    if det == "NFE":
        xstart, xend = 0,900
        xstep = (xend-xstart)/100
        xedges[det] = np.arange(xstart,xend,xstep)
        ystart, yend = 0,80
        ystep = (yend-ystart)/100
        yedges[det] = np.arange(ystart,yend,ystep)
    
    chunk_hist[det] = np.zeros(shape = (len(xedges[det])-1,
                                len(yedges[det])-1))
#     centers[det] = (xedges[det][:-1]+xedges[det][1:])/2
#     bin_sizes = xedges[det][1:] - xedges[det][:-1]    

histograms=[]

#iterating over dets
for det, code in trigger_ch.items():
    #Applying some inline cuts
    subchunk = data.query(f'trig_ch == {code}')
    subchunk[f'OFL_{det}_total_scaled'] = subchunk.loc[:,f'OFL_{det}_total'] * 100/auto_proc_high_energy[det]
    #Filling numpy arrays for histograms.
    ## ** (Maybe using weights inside the histogram.)

    histograms.append(dask_histogram2d(subchunk, f'OFL_{det}_total_scaled',
                                       f'OFL_chi2_{det}_total', bins=(xedges[det], yedges[det]), bins_range=None))

histograms = da.compute(*histograms)
chunk_hist = dict(zip(xedges.keys(), histograms))
  
real_min_c = {}
real_max_c = {}   
min_c = {}
max_c = {}
for det in trigger_ch.keys():
    real_min_c[det] = np.min(chunk_hist[det])
    real_max_c[det] = np.max(chunk_hist[det])
    chunk_hist[det] = scipy.signal.convolve2d(chunk_hist[det], np.ones((2,2)), mode='same')
    chunk_hist[det] = np.log(chunk_hist[det]+1)
    min_c[det] = np.min(chunk_hist[det])
    max_c[det] = np.max(chunk_hist[det])
fig,ax = plt.subplots(2,2,figsize=(16,16))
cmap = plt.get_cmap('viridis')
dets = trigger_ch.keys()
for count in range(data.npartitions):
    chunk = data.get_partition(count)[["trig_ch"] + [f'OFL_{det}_total' for det in dets] + [f'OFL_chi2_{det}_total' for det in dets]].compute()
    for det, code in trigger_ch.items(): 
        # For each det, we use a copy of the chunk to keep chunkes unchanged.
        subchunk = chunk.query(f'trig_ch == {code}')
        # Adjusting the copy
        subchunk.loc[:,f'OFL_{det}_total'] *= 100/auto_proc_high_energy[det]    
        i , j = (code-1)//2 , code%2 
        xidx = np.clip(np.digitize(subchunk[f'OFL_{det}_total'],xedges[det]), 0, chunk_hist[det].shape[0]-1)
        yidx = np.clip(np.digitize(subchunk[f'OFL_chi2_{det}_total'], yedges[det]), 0, chunk_hist[det].shape[1]-1)
        norm = plt.Normalize(min_c[det], max_c[det])
        s = norm(chunk_hist[det][xidx, yidx])
        c = cmap(s)
        s = 4 / (s + 0.1)
        # Scatter plots
        ax[i][j] = subchunk.plot.scatter(f'OFL_{det}_total', f'OFL_chi2_{det}_total', ax = ax[i,j],c=c,s=s)

for det,code in trigger_ch.items():
    i , j = (code-1)//2 , code%2
    
    norm = matplotlib.colors.LogNorm(real_min_c[det]+1, real_max_c[det]+1)
    fig.colorbar(plt.cm.ScalarMappable(norm=norm, cmap=cmap), ax=ax[i][j])
    ax[i,j].set_xlabel(f"OFL_{det}_total (uA)",fontsize = 20)
    ax[i,j].set_ylabel(f"OFL_chi2_{det}_total (uA)",fontsize = 20)
    ax[i,j].tick_params(axis='both', which='both', labelsize=15)
    ax[i][j].set_ylim([0,80])
    ax[i][j].set_xlim([0,900])

In [ ]:
# ***** Produce the heatmap too! ******

In [ ]:
data = RQReader(files,branches=["*OFL*","*trig_ch*",'*Integral*_total'],progressbar=True)
trigger_ch = {'NFC1':1,'NFH':2,'NFE':3,'NFC2':4}
KeV_bin = {'NFC1':22.9,'NFH':7.1,'NFE':5.7,'NFC2':17.1}
live_time = 44.85 #h
det_mass = 0.96 #g

data = RQReader(files,branches=["*OFL*","*trig_ch*",'*Integral*_total'],progressbar=True)
trigger_ch = {'NFC1':1,'NFH':2,'NFE':3,'NFC2':4}
#For different detectors, building histogram domains separately.
xedges = {}
yedges = {}
chunk_hist = {}
for det, code in trigger_ch.items():
    if det in ['NFC1','NFC2']:
        xstart, xend = 0,900
        xstep = (xend-xstart)/100
        xedges[det] = np.arange(xstart,xend,xstep)
        
        ystart, yend = 0,80
        ystep = (yend-ystart)/100
        yedges[det] = np.arange(ystart,yend,ystep)
        
    if det == "NFH":
        xstart, xend = 0,900
        xstep = (xend-xstart)/100
        xedges[det] = np.arange(xstart,xend,xstep)
        ystart, yend = 0,80
        ystep = (yend-ystart)/100
        yedges[det] = np.arange(ystart,yend,ystep)
    if det == "NFE":
        xstart, xend = 0,900
        xstep = (xend-xstart)/100
        xedges[det] = np.arange(xstart,xend,xstep)
        ystart, yend = 0,80
        ystep = (yend-ystart)/100
        yedges[det] = np.arange(ystart,yend,ystep)
    #2d emptry arrays for each channel.
    chunk_hist[det] = np.zeros(shape = (len(xedges[det])-1,
                                len(yedges[det])-1))
# iterating over the data
for chunk in data:
    #iterating over dets
    for det, code in trigger_ch.items():
        #Applying some inline cuts
        subchunk = chunk.query(f'trig_ch == {code}')
        subchunk.loc[:,f'OFL_{det}_total'] *= 100/auto_proc_high_energy[det]
        #Filling 2d numpy arrays.
        chunk_hist[det]=chunk_hist[det]+np.histogram2d(subchunk[f'OFL_{det}_total'],
                                        subchunk[f'OFL_chi2_{det}_total'],
                                        bins=(xedges[det], yedges[det]))[0]

fig,ax = plt.subplots(2,2,figsize=(16,16))
for det,code in trigger_ch.items():
    i , j = (code-1)//2 , code%2   
    axes = ax[i][j].pcolormesh(xedges[det],yedges[det], np.transpose(chunk_hist[det]),
                               cmap="viridis",norm=LogNorm())
    ax[i,j].set_xlabel(f"OFL_{det}_total (uA)",fontsize = 20)
    ax[i,j].set_ylabel(f"OFL_chi2_{det}_total (uA)",fontsize = 20)
    ax[i,j].tick_params(axis='both', which='both', labelsize=15)
    ax[i][j].set_ylim([0,80])
    ax[i][j].set_xlim([0,900])
    
    cbar = fig.colorbar(axes, ax = ax[i][j])
    cbar.ax.get_yaxis().labelpad = 15
    cbar.ax.set_ylabel('count', rotation=270,fontsize=15)
    cbar.ax.tick_params(axis='both', which='both', labelsize=15)
        

In [ ]:
trigger_ch = {'NFC1':1,'NFH':2,'NFE':3,'NFC2':4}
# Imran's work the same as Valentina's except NFH = 0.0488:
auto_proc_high_energy = {'NFE':0.103, 'NFH':0.049, 'NFC1':0.129,'NFC2':0.0796}

#making scatter plots
fig,ax = plt.subplots(2,2,figsize=(16,16))
for chunk in data:
    for det, code in trigger_ch.items(): 
        # For each det, we use a copy of the chunk 
        # to keep the main chunk unchanged to be used for the next det.
        subchunk = chunk.query(f'trig_ch == {code}')
        # Adjusting the copy
        subchunk.loc[:,f'OFL_{det}_total'] *= 100/auto_proc_high_energy[det]
        i , j = (code-1)//2 , code%2
        # Scatter plots
        ax[i][j] = subchunk.plot.scatter(f'OFL_{det}_total', f'OFL_chi2_{det}_total', ax = ax[i,j])

#Labels and notes.
for det,code in trigger_ch.items():
    i , j = (code-1)//2 , code%2
    ax[i][j].set_title(f'E = OFL_{det}_total*(100/{auto_proc_high_energy[det]})',fontsize = 15)
    ax[i][j].set_xlabel(f"Energy (ev)",fontsize = 20)
    ax[i][j].set_ylabel(f"OFL_chi2_{det}_total (uA)",fontsize = 20)
    ax[i][j].tick_params(axis='both', which='both', labelsize=15)
    ax[i][j].set_ylim([0,80])
    ax[i][j].set_xlim([0,900])

In [ ]:
data = RQReader(files,branches=["*OFL*","*trig*",'*Integral*','*file*','*Midas*'],progressbar=True)
trigger_ch = {'NFC1':1,'NFH':2,'NFE':3,'NFC2':4}
f_xhi2_a = {'NFE':0.00015,'NFH':.0005,'NFC1':.0005,'NFC2':.0005}
f_xhi2_b = {'NFE':1.25,'NFH':1.25,'NFC1':1.25,'NFC2':1.25}
f_xhi2_c = {'NFE':2,'NFH':2,'NFC1':2,'NFC2':2}
auto_proc_high_energy = {'NFE':0.103, 'NFH':0.049, 'NFC1':0.129,'NFC2':0.0796}


# Core of the cut, Using np.where to define energy limits. 

def Chi2_template(x,a,c,b):
#     x = np.where(x<600, a*x**c+b, float("inf"))
#     return x
    return a*x**c+b

# Instead of restarting the notebook. One can remove all cut definitions manually.
data.cutmanager.cuts ={}

trigger_ch = {'NFC1':1,'NFH':2,'NFE':3,'NFC2':4}
# defining cuts function in loops. Notice they all use Chi2_template.
# for det in trigger_ch.keys():
#     exec(f'''
# @registercut(branches=['OFL_{det}_total','OFL_chi2_{det}_total'])
# def chi2_cut_{det}(df):
#     energies = df['OFL_{det}_total']*100/auto_proc_high_energy['{det}']
#     chi2_limit = Chi2_template(energies,
#                                f_xhi2_a['{det}'],
#                                f_xhi2_c['{det}'],
#                                f_xhi2_b['{det}'])
#     return (df['OFL_chi2_{det}_total'] < chi2_limit)''')

In [ ]:
# *****registercut should recursively look for all internal functions! *****

# If we wanted to define cuts one by one

def Chi2_template(x,a,c,b):
    return a*x**c+b
@registercut(branches=['OFL_NFC1_total','OFL_chi2_NFC1_total'])
def chi2_cut_NFC1(df):
    energies = df['OFL_NFC1_total']*100/auto_proc_high_energy['NFC1']
    chi2_limit = Chi2_template(energies,
                               f_xhi2_a['NFC1'],
                               f_xhi2_c['NFC1'],
                               f_xhi2_b['NFC1'])
    return (df.OFL_chi2_NFC1_total < chi2_limit) | (energies > 600)
def Chi2_template(x,a,c,b):
    x = np.where(x<600, a*x**c+b, float("inf"))
    return x
@registercut
def chi2_cut_NFC2(df):
    energies = df['OFL_NFC2_total']*100/auto_proc_high_energy['NFC2']
    chi2_limit = Chi2_template(energies,
                               f_xhi2_a['NFC2'],
                               f_xhi2_c['NFC2'],
                               f_xhi2_b['NFC2']) 
    return (df['OFL_chi2_NFC2_total'] < chi2_limit)
@registercut
def chi2_cut_NFH(df):
    energies = df['OFL_NFH_total']*100/auto_proc_high_energy['NFH']
    chi2_limit = Chi2_template(energies,
                               f_xhi2_a['NFH'],
                               f_xhi2_c['NFH'],
                               f_xhi2_b['NFH']) 
    return (df['OFL_chi2_NFH_total'] < chi2_limit)
@registercut
def chi2_cut_NFE(df):
    energies = df['OFL_NFE_total']*100/auto_proc_high_energy['NFE']
    chi2_limit = Chi2_template(energies,
                               f_xhi2_a['NFE'],
                               f_xhi2_c['NFE'],
                               f_xhi2_b['NFE']) 
    return (df['OFL_chi2_NFE_total'] < chi2_limit)


In [ ]:
data.cutmanager.cuts ={}
def Chi2_template(x,a,c,b):
    return a*x**c+b
@registercut(branches=['OFL_NFC1_total','OFL_chi2_NFC1_total'])
def chi2_cut_NFC1(df):
    a = .0005
    b = 1.25
    c = 2
    first_eh_peak = 0.129
    energies = df['OFL_NFC1_total']*100/first_eh_peak
    chi2_limit = lambda x: a*x**c+b 
    return (df.OFL_chi2_NFC1_total < chi2_limit(energies)) | (energies > 600)

@registercut(branches=['OFL_NFC2_total','OFL_chi2_NFC2_total'])
def chi2_cut_NFC2(df):
    a = .0005
    b = 1.25
    c = 2
    first_eh_peak = 0.0796
    energies = df['OFL_NFC2_total']*100/first_eh_peak
    chi2_limit = lambda x: a*x**c+b 
    return (df.OFL_chi2_NFC2_total < chi2_limit(energies)) | (energies > 600)

@registercut(branches=['OFL_NFH_total','OFL_chi2_NFH_total'])
def chi2_cut_NFH(df):
    a = .0005
    b = 1.25
    c = 2
    first_eh_peak = 0.049
    energies = df['OFL_NFH_total']*100/first_eh_peak
    chi2_limit = lambda x: a*x**c+b 
    return (df.OFL_chi2_NFH_total < chi2_limit(energies)) | (energies > 600)

@registercut(branches=['OFL_NFE_total','OFL_chi2_NFE_total'])
def chi2_cut_NFE(df):
    a = .00015
    b = 1.25
    c = 2
    first_eh_peak = 0.103
    energies = df['OFL_NFE_total']*100/first_eh_peak
    chi2_limit = lambda x: a*x**c+b 
    return (df.OFL_chi2_NFE_total < chi2_limit(energies)) | (energies > 600)

    

In [ ]:
data.cutmanager.generate_cuts_py('HVeV13')

In [ ]:
data = RQReader(files,branches=[],progressbar=True)
data.attachcuts(['chi2_cut_NFC1'])
chunk = data.getchunk(0)
chunk.drop(chunk[chunk.chi2_cut_NFC1 == False ].index, inplace = True)
chunk

In [ ]:
data = RQReader(files,branches=["*OFL*","*trig*"],progressbar=True)
trigger_ch = {'NFC1':1,'NFH':2,'NFE':3,'NFC2':4}
# Imran's work the same as Valentina's except NFH = 0.0488:
auto_proc_high_energy = {'NFE':0.103, 'NFH':0.049, 'NFC1':0.129,'NFC2':0.0796}

fig,ax = plt.subplots(2,2,figsize=(16,16))
cuts = []

# Just a list(str) of cut names.
for det in trigger_ch.keys():
    cuts.append(f'chi2_cut_{det}')
# we attach all cuts to the data. We don't remove any entry. 
# cuts just appear as new columns.
data.attachcuts(cuts)
# Iterating the data

for chunk in data:
    for det, code in trigger_ch.items():
        # Working with subchunks to keep the main chunk unchanged.
        subchunk = chunk.query(f'trig_ch == {code}')
        # define colors for passing and rejected events.
        color = subchunk[f'chi2_cut_{det}'].apply(lambda cut: 'b' if cut else 'r')
        #adjusting the subchunk
        subchunk.loc[:,f'OFL_{det}_total'] *= 100/auto_proc_high_energy[det]
        i , j = (code-1)//2 , code%2
        #scatter plots for each chunk
        ax[i][j] = subchunk.plot.scatter(f'OFL_{det}_total', f'OFL_chi2_{det}_total', c= color,ax = ax[i,j])
# events passing        ax[i][j] = subchunk.plot.scatter(f'OFL_{det}_total', f'OFL_chi2_{det}_total',c ="b", ax = ax[i,j])
# events not passing    ax[i][j] = subchunk.plot.scatter(f'OFL_{det}_total', f'OFL_chi2_{det}_total',c="r", ax = ax[i,j])

# Making plots beautiful.
for det,code in trigger_ch.items():
    energies = np.linspace(0,900,1000)
    i , j = (code-1)//2 , code%2
    ax[i,j].axvline(x= 600, linewidth=2,ls = '-.', color='k',label = "OFL chi2 bound")
    ax[i][j].set_title(f'E = OFL_{det}_total*(100/{auto_proc_high_energy[det]})',fontsize = 15)
    ax[i][j].set_xlabel(f"Energy (ev)",fontsize = 20)
    ax[i][j].set_ylabel(f"OFL_chi2_{det}_total (uA)",fontsize = 20)
    ax[i][j].tick_params(axis='both', which='both', labelsize=15)
    ax[i,j].legend(loc="upper right", fontsize = 16)

In [ ]:
data = RQReader(files,branches=["*OFL*","*trig*"],progressbar=True)
trigger_ch = {'NFC1':1,'NFH':2,'NFE':3,'NFC2':4}
# Imran's work the same as Valentina's except NFH = 0.0488:
auto_proc_high_energy = {'NFE':0.103, 'NFH':0.049, 'NFC1':0.129,'NFC2':0.0796}
fig,ax = plt.subplots(2,2,figsize=(16,16))

cuts = []
# Just a list(str) of cut names.
for det in trigger_ch.keys():
    cuts.append(f'chi2_cut_{det}')
# we attach all cuts to the data. We don't remove any entry. 
# cuts just appear as new columns.
data.attachcuts(cuts)

# Iterating the data
for chunk in data:
    for det, code in trigger_ch.items():
        # Working with subchunks to keep the main chunk unchanged.
        subchunk = chunk.query(f'trig_ch == {code}')
        #adjusting the subchunk OFL --> energy    
        subchunk.loc[:,f'OFL_{det}_total'] *= 100/auto_proc_high_energy[det]
        # Define colors for different regions   
        def color_func(x):
            if x[f'OFL_{det}_total'] > 600:
                return 'k'
            elif x[f'chi2_cut_{det}'] == True:
                return 'b'
            else:
                return 'r'
        # build the color array. axis =1 is important!
        color = subchunk.apply(color_func,axis=1)
        i , j = (code-1)//2 , code%2
        #scatter plots for each chunk
        ax[i][j] = subchunk.plot.scatter(f'OFL_{det}_total', f'OFL_chi2_{det}_total', c= color,ax = ax[i,j])
        
# Making plots beautiful.
for det,code in trigger_ch.items():
    energies = np.linspace(0,900,1000)
    i , j = (code-1)//2 , code%2
    ax[i,j].axvline(x= 600, linewidth=2,ls = '-.', color='k',label = "OFL chi2 bound")
    ax[i][j].set_title(f'E = OFL_{det}_total*(100/{auto_proc_high_energy[det]})',fontsize = 15)
    ax[i][j].set_xlabel(f"Energy (ev)",fontsize = 20)
    ax[i][j].set_ylabel(f"OFL_chi2_{det}_total (uA)",fontsize = 20)
    ax[i][j].tick_params(axis='both', which='both', labelsize=15)
    ax[i,j].legend(loc="upper right", fontsize = 16)

In [ ]:
data = RQReader(files,branches=["*OFL*","*trig*"],progressbar=True)
trigger_ch = {'NFC1':1,'NFH':2,'NFE':3,'NFC2':4}
# Imran's work the same as Valentina's except NFH = 0.0488:
auto_proc_high_energy = {'NFE':0.103, 'NFH':0.049, 'NFC1':0.129,'NFC2':0.0796}

cuts = []
# Just a list(str) of cut names.
for det in trigger_ch.keys():
    cuts.append(f'chi2_cut_{det}')
# we attach all cuts to the data. We don't remove any entry. 
# cuts just appear as new columns.
data.attachcuts(cuts)
fig,ax = plt.subplots(2,2,figsize=(16,16))
# Iterating the data    
for chunk in data:
    for det, code in trigger_ch.items():
        # Working with subchunks to keep the main chunk unchanged.
        subchunk = chunk.query(f'trig_ch == {code}')
        # removing a a part of data this time.
        subchunk = subchunk.query(f'chi2_cut_{det} == True')
        #adjusting the subchunk OFL --> energy    
        subchunk.loc[:,f'OFL_{det}_total'] *= 100/auto_proc_high_energy[det]
        # Define colors for different regions
        def color_func(x):
            if x[f'OFL_{det}_total'] > 600:
                return 'k'
            elif x[f'chi2_cut_{det}'] == True:
                return 'b'
            else:
                return 'r'     
        # build the color array. axis =1 is important!
        color = subchunk.apply(color_func,axis=1)
        i , j = (code-1)//2 , code%2
        #scatter plots for each chunk
        ax[i][j] = subchunk.plot.scatter(f'OFL_{det}_total', f'OFL_chi2_{det}_total', c= color,ax = ax[i,j])

# Making plots beautiful.
for det,code in trigger_ch.items():
    energies = np.linspace(0,900,1000)
    i , j = (code-1)//2 , code%2
    ax[i,j].axvline(x= 600, linewidth=2,ls = '-.', color='k',label = "OFL chi2 bound")
    ax[i][j].set_title(f'E = OFL_{det}_total*(100/{auto_proc_high_energy[det]})',fontsize = 15)
    ax[i][j].set_xlabel(f"Energy (ev)",fontsize = 20)
    ax[i][j].set_ylabel(f"OFL_chi2_{det}_total (uA)",fontsize = 20)
    ax[i][j].tick_params(axis='both', which='both', labelsize=15)
    ax[i,j].legend(loc="upper right", fontsize = 16)
#     ax[i,j].set_yscale('log')

In [ ]:
data = RQReader(files,branches=["*OFL*","*trig*"],progressbar=True)
trigger_ch = {'NFC1':1,'NFH':2,'NFE':3,'NFC2':4}
# Imran's work the same as Valentina's except NFH = 0.0488:
auto_proc_high_energy = {'NFE':0.103, 'NFH':0.049, 'NFC1':0.129,'NFC2':0.0796}

fig,ax = plt.subplots(2,2,figsize=(16,16))
cuts = []

# Just a list(str) of cut names.
for det in trigger_ch.keys():
    cuts.append(f'chi2_cut_{det}')

# we attach all cuts to the data. We don't remove any entry. 
# cuts just appear as new columns.
data.attachcuts(cuts)
# Iterating the data
    
for chunk in data:
    for det, code in trigger_ch.items():
        # Working with subchunks to keep the main chunk unchanged.
        subchunk = chunk.query(f'trig_ch == {code}')
        subchunk = subchunk.query(f'chi2_cut_{det} == True')
        #adjusting the subchunk OFL --> energy    
        subchunk.loc[:,f'OFL_{det}_total'] *= 100/auto_proc_high_energy[det]
        # Define colors for different regions
        def color_func(x):
            if x[f'OFL_{det}_total'] > 600:
                return 'k'
            elif x[f'chi2_cut_{det}'] == True:
                return 'b'
            else:
                return 'r'   
    
        # build the color array. axis =1 is important!
        color = subchunk.apply(color_func,axis=1)
        
        i , j = (code-1)//2 , code%2
        #scatter plots for each chunk
        ax[i][j] = subchunk.plot.scatter(f'OFL_{det}_total', f'OFL_chi2_{det}_total', c= color,ax = ax[i,j])
# events passing        ax[i][j] = subchunk.plot.scatter(f'OFL_{det}_total', f'OFL_chi2_{det}_total',c ="b", ax = ax[i,j])
# events not passing    ax[i][j] = subchunk.plot.scatter(f'OFL_{det}_total', f'OFL_chi2_{det}_total',c="r", ax = ax[i,j])

# Making plots beautiful.
for det,code in trigger_ch.items():
    energies = np.linspace(0,900,1000)
    i , j = (code-1)//2 , code%2
    ax[i,j].axvline(x= 600, linewidth=2,ls = '-.', color='k',label = "OFL chi2 bound")
    ax[i][j].set_title(f'E = OFL_{det}_total*(100/{auto_proc_high_energy[det]})',fontsize = 15)
    ax[i][j].set_xlabel(f"Energy (ev)",fontsize = 20)
    ax[i][j].set_ylabel(f"OFL_chi2_{det}_total (uA)",fontsize = 20)
    ax[i][j].tick_params(axis='both', which='both', labelsize=15)
    ax[i,j].legend(loc="upper right", fontsize = 16)

In [ ]:
trigger_ch = {'NFC1':1,'NFH':2,'NFE':3,'NFC2':4}
auto_proc_high_energy = {'NFE':0.103, 'NFH':0.049, 'NFC1':0.129,'NFC2':0.0796}
data = RQReader(files,branches=["*OFL*","*trig*"],progressbar=True)

fig,ax = plt.subplots(2,2,figsize=(16,16))
# A string list of cut names.
cuts = []
for det in trigger_ch.keys():
    cuts.append(f'chi2_cut_{det}')
# Attaching cut values to each chunk. We get new columns for cuts.
data.attachcuts(cuts)
# iterate over the data
for chunk in data:
    for det, code in trigger_ch.items():
        # apply a cut
        subchunk = chunk.query(f'trig_ch == {code}')
        # define colors
        color = subchunk[f'chi2_cut_{det}'].apply(lambda cut: 'b' if cut else 'r')
        subchunk.loc[:,f'OFL_{det}_total'] *= 100/auto_proc_high_energy[det]
        i , j = (code-1)//2 , code%2
        # plotting here
        ax[i][j] = subchunk.plot.scatter(f'OFL_{det}_total', f'OFL_chi2_{det}_total', c= color,ax = ax[i,j])
# Add extra information to the plots (Nothing to do with chunks here.)
for det,code in trigger_ch.items():
    energies = np.linspace(0,900,1000)
    i , j = (code-1)//2 , code%2
    ax[i][j].plot(energies,Chi2_template(energies,
                                         f_xhi2_a[det],
                                         f_xhi2_c[det],
                                         f_xhi2_b[det]),
                  label=f'{f_xhi2_a[det]}*x**{f_xhi2_c[det]}+{f_xhi2_b[det]}',
                  ls = '--',
                  linewidth = 3,
                  c = 'k')
    ax[i,j].axvline(x= 600, linewidth=2,ls = '-.', color='k',label = "OFL chi2 bound")
    ax[i,j].legend(loc="lower right", fontsize = 16)
    ax[i][j].set_title(f'E = OFL_{det}_total*{(100/auto_proc_high_energy[det]):.2f}',fontsize = 15)
    ax[i][j].set_xlabel(f"Energy (eV)",fontsize = 20)
    ax[i][j].set_ylabel(f"OFL_chi2_{det}_total",fontsize = 20)
    ax[i][j].tick_params(axis='both', which='both', labelsize=15)
    ax[i][j].set_ylim([0,80])
    ax[i][j].set_xlim([0,900])
#     ax[i][j].set_yscale('log')

In [ ]:
data = RQReader(files,branches=["*OFL*","Integral_*_total","trig_ch"],progressbar=True)

In [ ]:
#scatter plot as explained above

trigger_ch = {'NFC1':1,'NFH':2,'NFE':3,'NFC2':4}

# Imran's work the same as Valentina's except NFH = 0.0488:
auto_proc_high_energy = {'NFE':0.103, 'NFH':0.049, 'NFC1':0.129,'NFC2':0.0796}


fig,ax = plt.subplots(2,2,figsize=(16,16))

cuts = []
for det in trigger_ch.keys():
    cuts.append(f'chi2_cut_{det}')
data.attachcuts(cuts)
for chunk in data:
    for det, code in trigger_ch.items():
        subchunk = chunk.query(f'trig_ch == {code}')
        color = subchunk[f'chi2_cut_{det}'].apply(lambda cut: 'b' if cut else 'r')
        subchunk.loc[:,f'OFL_{det}_total'] *= 100/auto_proc_high_energy[det]
        i , j = (code-1)//2 , code%2
        ax[i][j] = subchunk.plot.scatter(f'OFL_{det}_total',f'Integral_{det}_total',  c= color,ax = ax[i,j])
for det,code in trigger_ch.items():
    i , j = (code-1)//2 , code%2
    ax[i,j].axvline(x= 600, linewidth=2,ls = '-.', color='k',label = "OFL chi2 bound")
    ax[i][j].set_title(f'E = OFL_{det}_total*{(100/auto_proc_high_energy[det]):.2f}',fontsize = 15)
    ax[i][j].set_xlabel(f"Energy (eV)",fontsize = 20)
    ax[i][j].set_ylabel(f"Integral_{det}_total",fontsize = 20)
    ax[i][j].tick_params(axis='both', which='both', labelsize=15)
    ax[i,j].legend(loc="upper right", fontsize = 16)
#     ax[i][j].set_ylim([0,80])
#     ax[i][j].set_xlim([0,900])

In [ ]:
# A chunk sample
chunk = data.getchunk(0)
# chunk[chunk['chi2_cut_NFE']==True]
trigger_ch = {'NFC1':1,'NFH':2,'NFE':3,'NFC2':4}
det = 'NFC2'
code = trigger_ch[det]
chunk.query(f'trig_ch == {code} & chi2_cut_{det} == True')

In [ ]:
chunk = data.getchunk(0)
subchunk = chunk.query(f'trig_ch == {code}')

In [ ]:
# Similar to the very first cell. Check the documentation above.
data = RQReader(files,branches=["*OFL*","*trig*",'*Integral*'],progressbar=True)
trigger_ch = {'NFC1':1,'NFH':2,'NFE':3,'NFC2':4}

xedges = {}
global_hist = {}
chunk_hist_total = {}
chunk_hist = {}
centers= {}

for det, code in trigger_ch.items():
    if det in ['NFC1','NFC2']:
        start, end = 0,8000
        step = (end-start)/1000
        xedges[det] = np.arange(start,end,step)  
    if det == "NFH":
        start, end = 0,2500
        step = (end-start)/1000
        xedges[det] = np.arange(start,end,step)
    if det == "NFE":
        start, end = 0,2000
        step = (end-start)/1000
        xedges[det] = np.arange(start,end,step)
      
    global_hist[det] = np.zeros_like(xedges[det][:-1])
    chunk_hist[det] = np.zeros_like(xedges[det][:-1])
    chunk_hist_total[det] = np.zeros_like(xedges[det][:-1])
    centers[det] = (xedges[det][:-1]+xedges[det][1:])/2

cuts = []
for det in trigger_ch.keys():
    cuts.append(f'chi2_cut_{det}')
data.attachcuts(cuts)
    
for chunk in data:
    for det, code in trigger_ch.items():  
        
        global_hist[det]+=np.histogram(chunk[f'Integral_{det}_total'], bins=xedges[det])[0]
        
        subchunk = chunk.query(f'trig_ch == {code}')
        chunk_hist_total[det]+=np.histogram(subchunk[f'Integral_{det}_total'], bins=xedges[det])[0]
        
        subchunk = chunk.query(f'trig_ch == {code} & chi2_cut_{det} == True')
        chunk_hist[det]+=np.histogram(subchunk[f'Integral_{det}_total'], bins=xedges[det])[0]

fig,ax = plt.subplots(2,2,figsize=(16,16))
for det,code in trigger_ch.items():
    i , j = (code-1)//2 , code%2
#     ax[i,j].axvline(x= 600, linewidth=2,ls = '-.', color='k',label = "OFL chi2 bound")
    ax[i,j].step(centers[det],global_hist[det], label='All trigers')
    ax[i,j].step(centers[det],chunk_hist_total[det], label ='triggr cut')
    ax[i,j].step(centers[det],chunk_hist[det], label='triggr cut + Chi2 cut')
    ax[i,j].legend(loc="upper right", fontsize = 16)
    ax[i,j].set_xlabel(f"Integral_{det}_total (uA)",fontsize = 20)
    ax[i,j].set_ylabel("Events",fontsize = 20)
    ax[i,j].set_yscale('log')
    ax[i,j].tick_params(axis='both', which='both', labelsize=15)
    ax[i,j].set_xlim(left=0)
    ax[i,j].set_ylim(top=10e4)
    

In [ ]:
# Similar to the very first cell. Check the documentation above.
trigger_ch = {'NFC1':1,'NFH':2,'NFE':3,'NFC2':4}
xedges = {}
chunk_hist = {}
chunk_hist_total = {}
centers= {}
for det, code in trigger_ch.items():
    if det in ['NFC1','NFC2']:
        start, end = 0,1000
        step = (end-start)/500
        xedges[det] = np.arange(start,end,step)  
    if det == "NFH":
        start, end = 0,1000
        step = (end-start)/500
        xedges[det] = np.arange(start,end,step)
    if det == "NFE":
        start, end = 0,1000
        step = (end-start)/500
        xedges[det] = np.arange(start,end,step)
      
    chunk_hist[det] = np.zeros_like(xedges[det][:-1])
    chunk_hist_total[det] = np.zeros_like(xedges[det][:-1])
    centers[det] = (xedges[det][:-1]+xedges[det][1:])/2
            
cuts = []
for det in trigger_ch.keys():
    cuts.append(f'chi2_cut_{det}')
data.attachcuts(cuts)
for chunk in data:
    for det, code in trigger_ch.items():  
        subchunk = chunk.query(f'trig_ch == {code}')
        subchunk.loc[:,f'OFL_{det}_total'] *= 100/auto_proc_high_energy[det]
        chunk_hist_total[det]+=np.histogram(subchunk[f'OFL_{det}_total'], bins=xedges[det])[0]
        
        subchunk = chunk.query(f'trig_ch == {code} & chi2_cut_{det} == True')
        subchunk.loc[:,f'OFL_{det}_total'] *= 100/auto_proc_high_energy[det]
        chunk_hist[det]+=np.histogram(subchunk[f'OFL_{det}_total'], bins=xedges[det])[0]

fig,ax = plt.subplots(2,2,figsize=(16,16))
for det,code in trigger_ch.items():
    i , j = (code-1)//2 , code%2
    ax[i][j].set_title(f'E = OFL_{det}_total*{(100/auto_proc_high_energy[det]):.2f}',fontsize = 15)
    ax[i,j].axvline(x= 600, linewidth=2,ls = '-.', color='k',label = "OFL chi2 bound")
    ax[i,j].step(centers[det],chunk_hist[det], label='f_Chi2_cut')
    ax[i,j].step(centers[det],chunk_hist_total[det], label ='total',alpha = 0.5)
    ax[i,j].legend(loc="upper right", fontsize = 16)
    ax[i,j].set_xlabel(f"Energy (eV)",fontsize = 20)
    ax[i,j].set_ylabel("Events",fontsize = 20)
    ax[i,j].set_yscale('log')
    ax[i,j].tick_params(axis='both', which='both', labelsize=15)
    ax[i][j].set_xlim([0,800])

In [ ]:
# Similar to the very first cell. Check the documentation above.

trigger_ch = {'NFC1':1,'NFH':2,'NFE':3,'NFC2':4}

xedges = {}
chunk_hist = {}
chunk_hist_total = {}
centers= {}
for det, code in trigger_ch.items():
    if det in ['NFC1','NFC2']:
        start, end = 0,13000
        step = (end-start)/1000
        xedges[det] = np.arange(start,end,step)  
    if det == "NFH":
        start, end = 0,7500
        step = (end-start)/1000
        xedges[det] = np.arange(start,end,step)
    if det == "NFE":
        start, end = 0,2500
        step = (end-start)/1000
        xedges[det] = np.arange(start,end,step)
      
    chunk_hist[det] = np.zeros_like(xedges[det][:-1])
    chunk_hist_total[det] = np.zeros_like(xedges[det][:-1])
    centers[det] = (xedges[det][:-1]+xedges[det][1:])/2
            
# data.applycut('cExampleFuncCut')

cuts = []
for det in trigger_ch.keys():
    cuts.append(f'chi2_cut_{det}')
data.attachcuts(cuts)

for chunk in data:
    for det, code in trigger_ch.items():  
        subchunk = chunk.query(f'trig_ch == {code}')
#         color = subchunk[f'chi2_cut_{det}'].apply(lambda cut: 'b' if cut else 'r')
        subchunk.loc[:,f'OFL_{det}_total'] *= 100/auto_proc_high_energy[det]
        chunk_hist_total[det]+=np.histogram(subchunk[f'OFL_{det}_total'], bins=xedges[det])[0]
        
        subchunk = chunk.query(f'trig_ch == {code} & chi2_cut_{det} == True')
        subchunk.loc[:,f'OFL_{det}_total'] *= 100/auto_proc_high_energy[det]
        chunk_hist[det]+=np.histogram(subchunk[f'OFL_{det}_total'], bins=xedges[det])[0]

fig,ax = plt.subplots(2,2,figsize=(16,16))
for det,code in trigger_ch.items():
    i , j = (code-1)//2 , code%2
    ax[i,j].axvline(x= 600, linewidth=2,ls = '-.', color='k',label = "OFL chi2 bound")
    ax[i][j].set_title(f'E = OFL_{det}_total*{(100/auto_proc_high_energy[det]):.2f}',fontsize = 15)
    ax[i,j].step(centers[det],chunk_hist[det], label='f_Chi2_cut')
    ax[i,j].step(centers[det],chunk_hist_total[det], label ='total',alpha = 0.3)
    ax[i,j].legend(loc="upper right", fontsize = 16)
    ax[i,j].set_xlabel(f"Energy (eV)",fontsize = 20)
    ax[i,j].set_ylabel("Events",fontsize = 20)
    ax[i,j].set_yscale('log')
#     ax[i,j].set_xscale('log')
    ax[i,j].tick_params(axis='both', which='both', labelsize=15)
#     ax[i][j].set_xlim([0,300])

In [ ]:
chi2_cut_NFC1 = data.cutmanager.cuts['chi2_cut_NFC1'].cutfunc
chi2_cut_NFC1

In [ ]:
import inspect
# chi2_cut_NFC1
# inspect.getclosurevars(chi2_cut_NFC1)
inspect.getsource(chi2_cut_NFC1)

In [ ]:
data.cutmanager.generate_cuts_py('run13_4channel_OFL_chi2')